In [1]:
from keras.models import Model
from keras.layers import Input, multiply, Dense, Reshape, dot
import tensorflow as tf
from random import uniform
%run optimizers/Optimizer.ipynb
%run optimizers/initializers/DefaultInitializer.ipynb

class GradientDescent(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance,
                 numIterations=1000, initializer=DefaultInitializer()):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.numIterations = numIterations
        self.initializer = initializer
    
    def optimize(self, inputHandler, outputHandler):
        # TODO force type
        
        targetFeatures = inputHandler.peekNext()
        
        ai, ri, ti, si = self.initializer.initialize(targetFeatures,
                self.mean, self.pcaBasis, self.pcaVariance)

        # tensorflow placeholders for our training data
        train_mean = tf.placeholder(tf.float64, shape=self.mean.shape)
        train_pcaBasis = tf.placeholder(tf.float64, shape=self.pcaBasis.shape)
        train_pcaVariance = tf.placeholder(tf.float64, shape=self.pcaVariance.shape)
        
        while inputHandler.hasNext():
            
            targetFeatures = inputHandler.getNext()

            # tensorflow variables
            a, r, s, t, c = self._getVariables(ai, ri, ti, si)

            # model
            y_model = self._getTrainingModel(train_mean, train_pcaBasis, a, r, s, t, c)

            # errors
            error = self._getError(targetFeatures, y_model, a, train_pcaVariance)

            # optimizer
            optimizer = self._getOptimizer(error)

            # initialize values, create a session and run the model
            model = tf.global_variables_initializer()

            with tf.Session() as session:
                session.run(model)
                for _ in range(self.numIterations):
                    session.run(optimizer,
                                feed_dict={
                                    train_mean:self.mean,
                                    train_pcaBasis:self.pcaBasis,
                                    train_pcaVariance:self.pcaVariance
                                })

                return (session.run(a), session.run(r), session.run(t), session.run(s))
            # ai, ri, ti, si = ^ ^ ^ ^
    
    def _getVariables(self, ai, ri, ti, si):
        a = tf.Variable(ai, name='a')
        r = tf.Variable(ri, name='r')
        s = tf.Variable(si, name='s')
        t = tf.Variable(ti, name='t')
        ts = tf.tile(t, [1, self.numFeatures])
        ts = tf.reshape(ts, (2, self.numFeatures))
        c = tf.constant(np.array([[1.,0.,0.],[0.,1.,0.]]), name='c')
        return a, r, s, ts, c
    
    def _getTrainingModel(self, train_mean, train_pcaBasis, a, r, s, t, c):
        x = tf.matmul(train_pcaBasis, a)
        x = tf.add(train_mean, x)
        # Fortran Ordering!
        x = tf.transpose(tf.reshape(x,(self.numFeatures, 3)))
        x = tf.matmul(r, x)
        x = tf.matmul(c, x)
        x = (s * x) + (s * t)
        return tf.reshape(x, [-1])
    
    def _getError(self, targetFeatures, y_model, a, train_pcaVariance):
        e_lmk = tf.losses.mean_squared_error(targetFeatures, y_model)
        e_prior = tf.reduce_sum(tf.square(tf.divide(a, tf.sqrt(train_pcaVariance))))
        #e_t = tf.reduce_sum(tf.square(ts))

        return 0.33*tf.cast(e_lmk, tf.float64) + 0.67*tf.cast(e_prior, tf.float64)
    
    def _getOptimizer(self, error):
        return tf.train.AdamOptimizer(learning_rate=0.01).minimize(error)

        